In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import pandas as pd

In [4]:
# 크롬드라이버 열기 / 406에러 방지(지연)을 위한 헤더 추가
path = "chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36')
driver = webdriver.Chrome(executable_path=path,options=options)
driver.maximize_window()

# 리스트에 저장
melon_korean = []

# 드라이버가 해당 url 접속
for i in range(0, 10000, 50):
    driver.get(f'https://www.melon.com/new/index.htm#params%5BareaFlg%5D=I&po=pageObj&startIndex={i+1}')
    time.sleep(5)
    html = driver.page_source

    soup = bs(html, 'html.parser')
    
    #앨범코버, 제목, 가수 크롤해서 저장 
    covers = soup.find_all("a", {"class": "image_typeAll"})
    titles = soup.find_all("div", {"class": "ellipsis rank01"})
    artists = soup.find_all("span", {"class": "checkEllipsis"})

    for item in zip(covers, titles, artists):
        melon_korean.append(
            {
                'cover': item[0].find('img')["src"],
                'title': item[1].find('a').text,
                'artist': item[2].text,
            }
        )

#형태 변환
data = pd.DataFrame(melon_korean)

#크롬 종료
driver.quit()

#같은 데이터들 중복제거
tmp = data.drop_duplicates(['cover', 'title', 'artist'])

#csv파일로 저장
tmp.to_csv('melon_korean.csv', encoding='utf-8')